In [1]:
from tensorflow.keras.layers import Input, Conv2D, Dense, ReLU, Flatten, Softmax
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# Convert y_train into one-hot format
temp = []
for i in range(len(y_train)):
    temp.append(to_categorical(y_train[i], num_classes=10))
y_train = np.array(temp)
# Convert y_test into one-hot format
temp = []
for i in range(len(y_test)):    
    temp.append(to_categorical(y_test[i], num_classes=10))
y_test = np.array(temp)

In [4]:
#reshaping
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(1,3)(inputs)
out = ReLU()(out)
out = Flatten()(out)
out = Dense(10, activation=None)(out)
out = Softmax()(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 1)         10        
_________________________________________________________________
re_lu (ReLU)                 (None, 26, 26, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 676)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                6770      
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 6,780
Trainable params: 6,780
Non-trainable params: 0
___________________________________________________________

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 2s 829us/step - loss: 3.2919 - acc: 0.5537 - val_loss: 0.5674 - val_acc: 0.8258
Epoch 2/10
1875/1875 [==============================] - 1s 643us/step - loss: 0.4984 - acc: 0.8448 - val_loss: 0.4244 - val_acc: 0.8739
Epoch 3/10
1875/1875 [==============================] - 1s 664us/step - loss: 0.3538 - acc: 0.8937 - val_loss: 0.3560 - val_acc: 0.9000
Epoch 4/10
1875/1875 [==============================] - 1s 750us/step - loss: 0.3003 - acc: 0.9076 - val_loss: 0.3357 - val_acc: 0.9111
Epoch 5/10
1875/1875 [==============================] - 1s 617us/step - loss: 0.2814 - acc: 0.9146 - val_loss: 0.3184 - val_acc: 0.9075
Epoch 6/10
1875/1875 [==============================] - 1s 606us/step - loss: 0.2689 - acc: 0.9194 - val_loss: 0.3247 - val_acc: 0.9106
Epoch 7/10
1875/1875 [==============================] - 1s 633us/step - loss: 0.2581 - acc: 0.9198 - val_loss: 0.3020 - val_acc: 0.9187
Epoch 8/10
1875/1875 [==========================

In [9]:
X = X_test[0]
X.shape

(28, 28, 1)

In [10]:
model2 = Model(model.input, model.layers[-2].output)

In [11]:
y = model2.predict(X_test[[0]]) - model.weights[3].numpy()
y

array([[-12.546714 , -11.377294 ,  -9.379934 , -11.585654 , -11.749796 ,
        -17.505424 , -18.279175 ,  -0.4272291, -14.400676 , -10.785936 ]],
      dtype=float32)

In [12]:
in_json = {
    "in": X.astype(int).flatten().tolist(),
    "conv2d_weights": (model.weights[0].numpy()*(10**9)).round().astype(int).flatten().tolist(),
    "conv2d_bias": (model.weights[1].numpy()*(10**9)).round().astype(int).flatten().tolist(), # no need to sqaure the scaling because input was not scaled
    "dense_weights":(model.weights[2].numpy()*(10**9)).round().astype(int).flatten().tolist(),
    "dense_bias": np.zeros(model.weights[3].numpy().shape).tolist() # zero because we are not doing softmax in circom, just argmax
}

In [13]:
out_json = {
    "scale": 10**-18,
    "out": y.flatten().tolist(),
    "label": int(y.argmax())
}
out_json

{'scale': 1e-18,
 'out': [-12.546713829040527,
  -11.377293586730957,
  -9.379934310913086,
  -11.585654258728027,
  -11.749795913696289,
  -17.50542449951172,
  -18.2791748046875,
  -0.427229106426239,
  -14.400675773620605,
  -10.78593635559082],
 'label': 7}

In [14]:
import json

In [15]:
with open("mnist_input.json", "w") as f:
    json.dump(in_json, f)

In [16]:
with open("mnist_output.json", "w") as f:
    json.dump(out_json, f)